In [1]:
import numpy as np

## Define the ANN class

In [1]:
class ANN():
    def __init__(self,input_size=128*128*3,hidden_size=12,output_size=1):
        #initialize the weights
        self.a1,self.a2,self.z1,self.z2 = None,None,None,None
        self.w1 = np.random.randn(hidden_size,input_size)*0.01
        self.w2 = np.random.rand(output_size,hidden_size)*0.01
        #initialize the biases
        self.b1 = np.zeros((hidden_size,1))
        self.b2 = np.zeros((output_size,1))
        self.learning_rate = 0.01
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    def relu(self,z):
        return np.maximum(0,z)
    def relu_derivative(self,z):
        return np.where(z>0,1,0)
    def forward(self, x):
        self.z1 = np.dot(self.w1, x) + self.b1  # z1 = w1 * x + b1
        self.a1 = self.relu(self.z1)  # a1 = relu(z1)
        self.z2 = np.dot(self.w2, self.a1) + self.b2  # z2 = w2 * a1 + b2
        self.a2 = self.sigmoid(self.z2)  # a2 = sigmoid(z2)
        return self.a2
    def compute_loss(self,x,y):
        a2 = self.forward(x)
        loss = -y*np.log(a2)-(1-y)*np.log(1-a2)
        return loss
    def backward(self,x,y):
        m = y.shape[1]
        dZ2 = self.a2 - y
        dW2 = (1 / m) * np.dot(dZ2, self.a1.T)
        db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)
        dZ1 = np.dot(self.w2.T, dZ2) * self.relu_derivative(self.z1)
        dW1 = (1 / m) * np.dot(dZ1, x.T)
        db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)

        # Update parameters
        self.w1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.w2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
    def compute_loss(self, Y):
        m = Y.shape[1]
        logprobs = np.multiply(np.log(self.a2), Y) + np.multiply((1 - Y), np.log(1 - self.a2))
        loss = -np.sum(logprobs) / m
        return loss
    def fit(self,X,Y,epochs=1000,learning_rate=0.01):
        self.learning_rate = learning_rate
        for i in range(epochs):
            f=self.forward(X)
            loss=self.compute_loss(Y)
            self.backward(X,Y)
            if i%100==0:
                print(f'Loss at epoch {i} is {loss}')
    def predict(self, X, threshold=0.5):
        predictions = self.forward(X)
        return predictions > threshold
    

    
    

In [2]:
def reshape_data(train_inputs,train_outputs,test_inputs,test_outputs):
    train_inputs = np.array(train_inputs)
    train_inputs = np.array(train_inputs).reshape(train_inputs.shape[0], -1).T
    train_outputs = np.array(train_outputs).reshape(1, -1)
    test_inputs = np.array(test_inputs)
    test_inputs = np.array(test_inputs).reshape(test_inputs.shape[0], -1).T
    test_outputs = np.array(test_outputs)
    return train_inputs,train_outputs,test_inputs,test_outputs

In [3]:
import numpy as np

def with_MyAnn(train_inputs, train_outputs, test_inputs, test_outputs, output_names=['no_sepia', 'sepia']):
    # Reshape the data
    train_inputs, train_outputs, test_inputs, test_outputs = reshape_data(train_inputs, train_outputs, test_inputs, test_outputs)

    annTool = ANN(input_size=49152, hidden_size=23, output_size=1) 
    annTool.fit(train_inputs, train_outputs)
    
    computed_outputs = annTool.predict(test_inputs)  
    computed_outputs = computed_outputs[0]
    
    computed_outputs=[1 if i==True else 0 for i in computed_outputs]
    return computed_outputs


In [4]:
def run_myann_experiments(train_inputs, train_outputs, test_inputs, test_outputs):
    configurations = [
        {'epochs': 500, 'learning_rate': 0.01, 'hidden_size': 12},
        {'epochs': 1000, 'learning_rate': 0.005, 'hidden_size': 12},
        {'epochs': 500, 'learning_rate': 0.01, 'hidden_size': 24},
        {'epochs': 1000, 'learning_rate': 0.005, 'hidden_size': 24},
        {'epochs': 500, 'learning_rate': 0.01, 'hidden_size': 36},
        {'epochs': 1000, 'learning_rate': 0.005, 'hidden_size': 36}
    ]
    train_inputs, train_outputs, test_inputs, test_outputs = reshape_data(train_inputs, train_outputs, test_inputs, test_outputs)
    results = []
    for config in configurations:
        model = ANN(input_size=train_inputs.shape[0], hidden_size=config['hidden_size'], output_size=train_outputs.shape[0])
        print(f"Training with config: epochs={config['epochs']}, learning_rate={config['learning_rate']}, hidden_size={config['hidden_size']}")
        model.fit(train_inputs, train_outputs, epochs=config['epochs'], learning_rate=config['learning_rate'])
        predictions = model.predict(test_inputs)
        accuracy = np.mean((predictions == test_outputs).astype(int))  # Simple accuracy calculation
        results.append({'config': config, 'accuracy': accuracy})
        print(f"Accuracy: {accuracy}")
    
    return results
